# Histopathologic Cancer Detection

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers
import pandas
import os
import cv2
import numpy as np
import random
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split

In [ ]:
TRAIN_PATH = "./dataset/train/"
TEST_PATH = "./dataset/test/"
IMG_WIDTH, IMG_HEIGHT = 32, 32
# Between 1 and 220025
NUMBER_OF_SAMPLES = 200000
LOAD_MATRICES = True

In [ ]:
df = pandas.read_csv("./dataset/train_labels.csv")
df.head(3)

### Check if null
We should make sure that there are no `null` fields in the dataset.
If there were any:
* Remove data row
* Fill according to other data (Which can not be done here, as the data are images)

In [ ]:
number_of_empty_id_rows = len(df[df.id.isnull()])
number_of_empty_label_rows = len(df[df.label.isnull()])

if number_of_empty_id_rows == 0:
    print("[NOTE]: Dataset 'id' column has no 'Null' values. No need to fill/remove rows.")
if number_of_empty_label_rows == 0:
    print("[NOTE]: Dataset 'label' column has no 'Null' values. No need to fill/remove rows.")

### Understanding dataset

* Sampling positive and negative samples will help us understand the dataset.

In [ ]:
positive_tumor_samples = df[df.label == 1]
number_of_posive_tumor_samples = len(positive_tumor_samples)
number_of_posive_tumor_samples
print(f"Number of positive tumor samples: {number_of_posive_tumor_samples}")

In [ ]:
negative_tumor_samples = df[df.label == 0]
number_of_negative_tumor_samples = len(negative_tumor_samples)
number_of_negative_tumor_samples
print(f"Number of negative tumor samples: {number_of_negative_tumor_samples}")

## Data Information provided by [Kaggle](https://www.kaggle.com/c/histopathologic-cancer-detection/data)

In this dataset, you are provided with a large number of small pathology images to classify. Files are named with an image id. The `train_labels.csv` file provides the ground truth for the images in the train folder. You are predicting the labels for the images in the test folder. ***A positive label indicates that the center `32x32px` region of a patch contains at least one pixel of tumor tissue.*** Tumor tissue in the outer region of the patch ***does not*** influence the label. This outer region is provided to enable fully-convolutional models that do not use zero-padding, to ensure consistent behavior when applied to a whole-slide image.

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(20, 8))

fig.suptitle('Histopathologic scans of lymph node sections',fontsize=20)

ax[0, 0].set_ylabel("Normal Tissue Samples", size='large')
ax[1, 0].set_ylabel("Positive Tumor Samples", size='large')

for i in range(5):
    # Negative Tumor Tissue Sample
    random_negative_sample = negative_tumor_samples.iloc[random.randint(0, number_of_negative_tumor_samples)]
    random_negative_sample_id = random_negative_sample.id
    random_negative_sample_label = random_negative_sample.label
    random_negative_sample_path = os.path.join(TRAIN_PATH, random_negative_sample_id) + '.tif'
    random_negative_img = cv2.imread(random_negative_sample_path)
    random_negative_img = cv2.cvtColor(random_negative_img, cv2.COLOR_BGR2RGB)
    rect = patches.Rectangle((32, 32), 32, 32, linewidth=3, edgecolor='b', facecolor='none')
    ax[0, i].add_patch(rect)
    ax[0, i].imshow(random_negative_img)
    
    # Positive Tumor Tissue Samples
    random_positive_sample = positive_tumor_samples.iloc[random.randint(0, number_of_posive_tumor_samples)]
    random_positive_sample_id = random_positive_sample.id
    random_positive_sample_label = random_positive_sample.label
    random_positive_sample_path = os.path.join(TRAIN_PATH, random_positive_sample_id) + '.tif'
    random_positive_img = cv2.imread(random_positive_sample_path)
    random_positive_img = cv2.cvtColor(random_positive_img, cv2.COLOR_BGR2RGB)
    rect = patches.Rectangle((32, 32), 32, 32, linewidth=3, edgecolor='r', facecolor='none')
    ax[1, i].imshow(random_positive_img)
    ax[1, i].add_patch(rect)

In [ ]:
df.head(10)

In [ ]:
def read_convert_images_from_df(dataframe, base_dir):
    img_list = []
    img_label_list = []
    total_number_of_data = len(dataframe)
    for index, img_datum in dataframe.iterrows():
        if index < NUMBER_OF_SAMPLES:
            print(f"{index + 1} out of {total_number_of_data}")
            print(f"Image ID: {img_datum.id}, Label: {img_datum.label}")
            img_id = img_datum.id
            img_label = df[df.id == img_id].iloc[0].label
            img_path = os.path.join(base_dir, img_id) + '.tif'
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            img = np.asarray(img, dtype=float)
            img /= 255.0
            img_list.append(img)
            img_label_list.append(img_label)
    return img_list, img_label_list

In [ ]:
def read_convert_images_from_path(path):
    img_list = []
    img_id_list = []
    for idx, img_name in enumerate(os.listdir(path)):
        print(f"{idx + 1} out of {len(os.listdir(path))}")
        img_id = img_name.split(".tif")[0]
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        img = np.asarray(img, dtype=float)
        img /= 255.0
        img_list.append(img)
        img_id_list.append(img_id)
    img_list = np.asarray(img_list, dtype='float')
    img_id_list = np.asarray(img_id_list)
    return img_list, img_id_list

In [ ]:
def load_matrices():
    X = np.load('./matrices/X_100k.npy')
    y = np.load('./matrices/y_100k.npy')
    
    X_eval = np.load('./matrices/X_EVAL.npy')
    ID_eval = np.load('./matrices/ID_EVAL.npy')
    
    return X, y, X_eval, ID_eval

In [ ]:
if LOAD_MATRICES:
    X, y, X_eval, ID_eval = load_matrices()
    print(f"X Shape: {X.shape}")
    print(f"y Shape: {y.shape}")
    print(f"X eval Shape: {X_eval.shape}")
    print(f"ID eval Shape: {ID_eval.shape}")
else:
    X, y = read_convert_images_from_df(df, TRAIN_PATH)
    
    X_eval, ID_eval = read_convert_images_from_path(TEST_PATH)
    
    X = np.asarray(X, dtype=float)
    y = np.asarray(y)
    
    np.save('./matrices/X_100k.npy', X)
    np.save('./matrices/y_100k.npy', y)
    
    np.save('./matrices/X_EVAL.npy', X_eval)
    np.save('./matrices/ID_EVAL.npy', id_list)
    
    print("X saved!")
    print("y saved!")
    
    print("X eval saved!")
    print("ID eval saved!")
    
    print(f"X Shape: {X.shape}")
    print(f"y Shape: {y.shape}")
    
    print(f"X eval Shape: {X_evall.shape}")
    print(f"ID eval Shape: {ID_eval.shape}")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [ ]:
print(f"X Train shape: {x_train.shape}\ny train Shape: {x_train.shape}\nX Test shape: {x_test.shape}\ny test shape: {y_test.shape}")

In [ ]:
vgg19 = VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(32, 32, 3),
    pooling=None,
    classes=2,
    classifier_activation="sigmoid",
)
model = tf.keras.models.Sequential(vgg19.layers[:10])
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [ ]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=64, epochs=50, callbacks=[tensorboard_callback])

In [ ]:
def predict(x_test):
    y_pred = (model.predict(x_test) > 0.5).astype("int32")
    return y_pred

In [ ]:
y_pred_eval = predict(X_eval)

In [ ]:
if os.path.exists('./submission.csv'):
    !rm submission.csv
    print("Removed old version.")

with open('./submission.csv', 'w+') as f:
    f.write("id,label\n")
    for idx, y in enumerate(y_pred_eval):
        f.write(f'{id_list[idx]},{y[0]}\n')
    print("Submission.csv file created!")

In [ ]:
!./submit_csv.sh 100k-training-sample-10layerVGG19